In [1]:
import os
import sys
import math as m
import pandas as pd
import csv
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm

plt.style.use('ggplot')

In [2]:
## Import data for Shanghai
fang_chengdu_pt = pd.read_csv("/Users/mjfoster/Dropbox (MIT)/STL_WeChatProject/1_shapefiles/fang/fang_pruned_residential_CH_jan11_gcj02.csv",encoding="utf-8")
fang_chengdu = pd.read_csv("/Users/mjfoster/Dropbox (MIT)/STL_WeChatProject/1_ShapeFiles/working/fang_jan13/fang_chengdu_aggregated_residential.csv",encoding="utf-8")
landscan_chengdu = pd.read_csv("/Users/mjfoster/Dropbox (MIT)/STL_WeChatProject/1_ShapeFiles/China_LandScan2014/GCJ02/chengdu_landscan_2014_corrected_gcj02.csv",encoding="utf-8")
# LS = pd.read_csv(")

In [3]:
# Make sure everything imported correctly for Fang
print fang_chengdu.columns
fang_chengdu.head()

Index([u'pointid', u'grid_code', u'population', u'distance_m', u'PNTCNT',
       u'avg_price_mean', u'max_price_mean', u'min_price_mean',
       u'avg_size_mean', u'max_size_mean', u'avg_psm_mean'],
      dtype='object')


,pointid,grid_code,population,distance_m,PNTCNT,avg_price_mean,max_price_mean,min_price_mean,avg_size_mean,max_size_mean,avg_psm_mean
0,10753007,305,305,57398.170929,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10753008,168,168,56771.422870,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10753006,33,33,58029.179006,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10753044,202,202,38723.238920,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10753043,200,200,39050.786405,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Remove null values from Fang, fill with zero
fang_chengdu = fang_chengdu.fillna(0)
print fang_chengdu.columns
fang_chengdu.head()

Index([u'pointid', u'grid_code', u'population', u'distance_m', u'PNTCNT',
       u'avg_price_mean', u'max_price_mean', u'min_price_mean',
       u'avg_size_mean', u'max_size_mean', u'avg_psm_mean'],
      dtype='object')


,pointid,grid_code,population,distance_m,PNTCNT,avg_price_mean,max_price_mean,min_price_mean,avg_size_mean,max_size_mean,avg_psm_mean
0,10753007,305,305,57398.170929,0,0,0,0,0,0,0
1,10753008,168,168,56771.422870,0,0,0,0,0,0,0
2,10753006,33,33,58029.179006,0,0,0,0,0,0,0
3,10753044,202,202,38723.238920,0,0,0,0,0,0,0
4,10753043,200,200,39050.786405,0,0,0,0,0,0,0


In [5]:
# Make sure LandScan imported correctly
print landscan_chengdu.columns
landscan_chengdu.head()

Index([u'pointid', u'grid_code', u'population', u'distance_m'], dtype='object')


,pointid,grid_code,population,distance_m
0,10753007,305,305,57398.170929
1,10753008,168,168,56771.422870
2,10753006,33,33,58029.179006
3,10753044,202,202,38723.238920
4,10753043,200,200,39050.786405


In [6]:
## Histogram of Logged Landscan count
fig = plt.figure()
ax = fig.add_subplot(111)
ax.hist([m.log(x+1) for x in landscan_chengdu.population],bins=30)

ax.set_xlabel('Grid IDs from Landcsan')
ax.set_ylabel('Normalized count on grid')
ax.set_title('Logged Landscan population per grid')
plt.tight_layout()
plt.show()

/Users/mjfoster/anaconda/lib/python2.7/site-packages/matplotlib/tight_layout.py:222: UserWarning: tight_layout : falling back to Agg renderer
  warnings.warn("tight_layout : falling back to Agg renderer")


In [7]:
## Histogram of Logged Fang listing count (run again using Fang prices)
fig = plt.figure()
ax = fig.add_subplot(111)
ax.hist([m.log(x+1) for x in fang_chengdu.PNTCNT],bins=30)
# ax.hist(weibo_SH.sum_checkin,bins=20)

ax.set_xlabel('Grid IDs from Landcsan')
ax.set_ylabel('Normalized count on grid')
ax.set_title('Logged Fang listings per grid')
plt.tight_layout()
plt.show()

In [8]:
##  Show scatter plots of original weibo and landscan side by side by ID
fig = plt.figure()
ax = fig.add_subplot(111)
ax.scatter(landscan_chengdu.pointid, [x/float(max(landscan_chengdu.population)) for x in landscan_chengdu.population],marker=',',c='r',s =18)
ax.scatter(fang_chengdu.pointid, [x/float(max(fang_chengdu.PNTCNT)) for x in fang_chengdu.PNTCNT],marker = 'o',c='b',s = 18)

ax.set_xlabel('ID')
ax.set_ylabel('Frequency')
ax.set_title('Raw Fang and Landscan Counts by ID ')
plt.tight_layout()
plt.show()

In [9]:
##  Show scatter plots of original weibo and landscan side by side by Distance from City Center
fig = plt.figure()
ax = fig.add_subplot(111)
ax.scatter(landscan_chengdu.distance_m, [x/float(max(landscan_chengdu.population)) for x in landscan_chengdu.population],marker=',',c='r',s =18)
ax.scatter(fang_chengdu.distance_m, [x/float(max(fang_chengdu.PNTCNT)) for x in fang_chengdu.PNTCNT],marker = 'o',c='b',s = 18)

ax.set_xlabel('ID')
ax.set_ylabel('Frequency')
ax.set_title('Raw Fang and Landscan Counts by Distance from City Center')
plt.tight_layout()
plt.show()

In [10]:
##  Show scatter plots of z-scores of Fang and LandScan
from scipy import stats

fig = plt.figure()
ax = fig.add_subplot(111)
ax.scatter(landscan_chengdu.pointid, stats.zscore(landscan_chengdu.population),marker=',',c='r',s =18)
ax.scatter(fang_chengdu.pointid, stats.zscore(fang_chengdu.PNTCNT),marker = 'o',c='b',s = 18)

ax.set_xlabel('ID')
ax.set_ylabel('Z-Score')
ax.set_title('Scatter Plot of Z-Scores for Fang and Landscan')
plt.tight_layout()
plt.show()


# plt.scatter(weibo_SH.ID, weibo_SH.sum_checkin)
# plt.show()

In [11]:
## Correlation analysis for raw data
corr_CD = np.corrcoef(landscan_chengdu.sort('pointid', ascending=True).population, fang_chengdu.sort('pointid', ascending=True).PNTCNT)
print corr_CD

[[ 1.          0.30454277]
 [ 0.30454277  1.        ]]


/Users/mjfoster/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


In [12]:
## Correlation analysis for LOGGED data
corr_CD = np.corrcoef([m.log(x+1) for x in landscan_chengdu.sort('pointid', ascending=True).population], [m.log(x+1) for x in fang_chengdu.sort('pointid', ascending=True).PNTCNT])
print corr_CD

[[ 1.          0.40096225]
 [ 0.40096225  1.        ]]


/Users/mjfoster/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


In [13]:
## Find "subtraction" between Landscan and Weibo z-scores of LOGGED counts

fig = plt.figure()
ax = fig.add_subplot(111)

ax.scatter(landscan_chengdu.pointid,stats.zscore([m.log(x+1) for x in landscan_chengdu.population])- stats.zscore([m.log(x+1) for x in fang_chengdu.PNTCNT]),marker=',',c='r',s =18)

ax.set_xlabel('Grid IDs from Landcsan')
ax.set_ylabel('Diff of Normalized Counts')
ax.set_title('"Subtraction" between Fang and Landscan Z-Scores counts by grid')
plt.tight_layout()
plt.show()

In [16]:
## Export Results of Z-score and subtraction analysis

results_CD = landscan_chengdu
results_CD['fang_count'] = fang_chengdu['PNTCNT']
results_CD['zscore_Landscan']=stats.zscore(landscan_chengdu.population)
results_CD['zscore_ln_Landscan']=stats.zscore([m.log(x+1) for x in landscan_chengdu.population])
results_CD['zscore_Fang']=stats.zscore([m.log(x+1) for x in fang_chengdu.PNTCNT])
results_CD['zscore_ln_Fang']=stats.zscore(fang_chengdu.PNTCNT)


results_CD['diff'] = results_CD['zscore_Landscan']-results_CD['zscore_Fang']
results_CD['diff_ln'] = results_CD['zscore_ln_Landscan']-results_CD['zscore_ln_Fang']
results_CD.head()

results_CD.to_csv('/Users/mjfoster/Dropbox (MIT)/STL_WeChatProject/1_ShapeFiles/working/fang_jan13/fang_landscan_results_CD_counts.csv',encoding="utf-8")

In [ ]:
print "notebook complete"